In [20]:
import time

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, LeakyReLU, Flatten
from tensorflow.keras.layers import Dense, Conv2DTranspose, Reshape, Dropout
from tensorflow.keras.models import Sequential

import numpy as np
import matplotlib.pyplot as plt

In [21]:
def make_generator():
  model = Sequential()
  model.add(Dense(4*4*512, input_dim = 128+54))
  model.add(ReLU())
  
  model.add(Reshape((4,4,512)))
  model.add(Conv2DTranspose(256, (5, 5), strides=(2,2), padding='same'))
  model.add(ReLU())
            
  model.add(Conv2DTranspose(128, (5, 5), strides=(2,2), padding='same'))
  model.add(ReLU())
  
  model.add(Conv2DTranspose(64, (5, 5), strides=(2,2), padding='same'))
  model.add(ReLU())
  
  model.add(Conv2DTranspose(1, (5, 5), strides=(2,2), padding='same', activation='sigmoid'))
            
  return model

In [22]:
def make_discriminator():
  model = Sequential()
  model.add(Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=(64,64,1)))
  model.add(LeakyReLU(alpha=0.2))
  
  model.add(Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  
  model.add(Conv2D(256, (5, 5), strides=(2, 2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2D(512, (5, 5), strides=(2, 2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  
  model.add(Flatten())
  model.add(Dense(1))
  return model

In [23]:
@tf.function
def gen_fake_image(z):
    image_sample = generator(z)
    return tf.cast(image_sample, dtype = tf.dtypes.float32)

In [24]:
@tf.function
def cal_gradient_penalty(real_data, z):
  fake_data = gen_fake_image(z)

  alpha = tf.random.uniform(
        shape=[BATCH_SIZE,1], 
        minval=0.,
        maxval=1.
    )

  differences = fake_data - real_data
  interpolates = real_data + tf.reshape((alpha*tf.reshape(differences, (BATCH_SIZE,-1))), (BATCH_SIZE, 64, 64, 1))
  gradients = tf.gradients(discriminator(interpolates), [interpolates])[0]
  slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients), axis = 1))
  gradient_penalty = tf.reduce_mean((slopes-1.)**2)
  return gradient_penalty

In [25]:
@tf.function
def cal_disc_cost(real_data, z, gradient_penalty):
  fake_data = gen_fake_image(z)
  return tf.reduce_mean(discriminator(fake_data)) - tf.reduce_mean(discriminator(real_data)) + LAMBDA*gradient_penalty

In [26]:
@tf.function
def make_one_hot(c, num, seed = None, num_char = 54):
  return tf.concat([tf.reshape(tf.tile(tf.one_hot(c, num_char, dtype = tf.dtypes.float32), [num]), [-1, num_char]),
                    tf.random.normal(shape = (num,128), dtype = tf.dtypes.float32, seed = seed)], axis = 1)

In [27]:
@tf.function
def make_consist(num_char = 54):
  z = tf.tile(tf.random.normal(shape = (1,128), dtype = tf.dtypes.float32), [num_char,1])
  letter_one_hot = tf.reshape(tf.one_hot(0, num_char, dtype = tf.dtypes.float32), [1,-1])
  for c in range(1, num_char):
    temp = tf.reshape(tf.one_hot(c, num_char, dtype = tf.dtypes.float32), [1, -1])
    letter_one_hot = tf.concat([letter_one_hot, temp], axis = 0)
  return tf.concat([letter_one_hot,z], axis = 1)

In [28]:
def update_disc(batch_data, c):
  real_data = tf.cast(batch_data, dtype = tf.float32)
  tf.random.set_seed(None)
  z = make_one_hot(c, BATCH_SIZE)

  with tf.GradientTape() as tape:
    gradient_penalty = cal_gradient_penalty(real_data, z)
    disc_cost = cal_disc_cost(real_data, z, gradient_penalty)
  disc_gradient = tape.gradient(disc_cost, discriminator.trainable_variables)
  disc_opt.apply_gradients(zip(disc_gradient, discriminator.trainable_variables))
  return disc_cost

In [29]:
def update_gen(c):
    tf.random.set_seed(None)
    z = make_one_hot(c, BATCH_SIZE)
    with tf.GradientTape() as tape:
      fake_data = gen_fake_image(z)
      gen_cost = -tf.reduce_mean(discriminator(fake_data))
    gen_gradient = tape.gradient(gen_cost, generator.trainable_variables)
    gen_opt.apply_gradients(zip(gen_gradient, generator.trainable_variables))
    return gen_cost

In [30]:
def plot_image(iteration, numchar = 54):
    z = make_consist(num_char = numchar)
    fig = plt.figure(figsize=(9,6))
    image_sample = gen_fake_image(z)
    image_sample = image_sample.numpy().reshape(-1,64,64,1)
    for i in range(image_sample.shape[0]):
      plt.subplot(9, 6, i+1)
      plt.axis('off')
      plt.imshow(image_sample[i, :, :, 0], cmap = 'gray')
      plt.savefig('save_img/iteration{}.png'.format(iteration))
    plt.show()
    plt.close()
    
   
def plot_loss(disc_cost_epoch, gen_cost_epoch):
  fig, axes = plt.subplots(1,2, figsize = (16,4))
  axes[0].plot(disc_cost_epoch, label = 'discriminator loss', color = 'orange')
  axes[0].legend()
  
  axes[1].plot(gen_cost_epoch, label = 'generator loss', color = 'blue')
  axes[1].legend()
  plt.show()
  plt.clf()


def plot_same(z_fix, iteration):
    fig = plt.figure(figsize=(9,6))
    image_sample = gen_fake_image(z_fix)
    image_sample = image_sample.numpy().reshape(-1,64,64,1)
    for i in range(image_sample.shape[0]):
      plt.subplot(9, 6, i+1)
      plt.axis('off')
      plt.imshow(image_sample[i, :, :, 0], cmap = 'gray')
      plt.savefig('save_img_fix/iteration{}.png'.format(iteration))
    plt.show()
    plt.close()

In [31]:
disc_opt = tf.keras.optimizers.Adam(
        learning_rate=1e-4,
        beta_1=0.5, 
        beta_2=0.99)

gen_opt = tf.keras.optimizers.Adam(
        learning_rate=1e-4,
        beta_1=0.5, 
        beta_2=0.99)

In [32]:
# z_fix = make_consist(num_char = 54)
# np.save('z_fix.npy', z_fix.numpy())

z_fix = tf.constant(np.load('z_fix.npy'))

In [33]:
data = np.load('fontarray.npy')
data = data/255.0
data = data.reshape(-1,54,64,64,1)
data = data.transpose(1,0,2,3,4)

In [34]:
generator = make_generator()
discriminator = make_discriminator()

In [ ]:
#กขฃฅคฆงจฉชซฌญฎฏฐฑฒณดตถทธนบปผฝพฟภมยรลวศษสหฬอฮ๐๑๒๓๔๕๖๗๘๙

BATCH_SIZE = 64 # Batch size
CRITIC_ITERS = 5 # For WGAN and WGAN-GP, number of critic iters per gen iter
LAMBDA = 10 # Gradient penalty lambda hyperparameter
ITERS = 2500 # How many generator iterations to train for 
OUTPUT_DIM = (64,64,1) # Number of pixels 
sample_interval = 10

gen_cost_epoch = []
disc_cost_epoch = []

for iteration in range(ITERS):
    start = time.time()
    for c in range(54):
      data_iter = tf.data.Dataset.from_tensor_slices(data[c]).shuffle(buffer_size = 100).batch(BATCH_SIZE, drop_remainder=True).repeat()
      data_batch = iter(data_iter)
      gen_cost_c = []
      for i in range(CRITIC_ITERS):
        disc_cost_batch = []
        disc_cost_batch.append(update_disc(next(data_batch),c).numpy())
        # print('finish update c of char:{}, epoch:{}'.format(c, iteration))
      gen_cost_c.append(update_gen(c).numpy())
#       print('finish update gen of char:{}, epoch:{}'.format(c, iteration))
    gen_cost_epoch.append(np.mean(gen_cost_c))
    disc_cost_epoch.append(tf.reduce_mean(disc_cost_batch))
    print('finish epoch:{} --- time use this epoch:{}'.format(iteration, time.time() - start))
    if iteration % sample_interval == 0:
        generator.save('checkpoint_gen/cp_gen_{}th_epoch.h5'.format(iteration))
        discriminator.save('checkpoint_disc/cp_disc_{}_th_epoch.h5'.format(iteration))
        plot_image(iteration)
        plot_same(z_fix, iteration)
        plot_loss(disc_cost_epoch, gen_cost_epoch)